<a href="https://colab.research.google.com/github/benjaminnigjeh/keyProteoforms/blob/main/classicQuantification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install required libraries

In [ ]:
!apt-get update
!apt-get install -y mono-complete
!pip install fisher-py

#Import external libraries

In [ ]:
from fisher_py.data.business import Scan
from fisher_py import RawFile
import re
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import pickle
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.stats import linregress
from sklearn.metrics.pairwise import cosine_similarity

#DataBank functions

In [ ]:
def wholeCasting(folder_path, cast_path):
    os.chdir(folder_path)

    def helper_regex(text):
        match = re.search(rf"{'Full'}\s+(\w+)", text)
        if match:
            return match.group(1)
        return None
    def find_matching_keys(sequence: str, substring_dict: dict) -> list:
        return [key for key, substrings in substring_dict.items() if any(substring in sequence for substring in substrings)]


    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    substring_dict_sample = {"Disease": ["PDAD", "AD", "PD"], "Normal Healthy": ["NC"]}
    substring_dict_prep = {"Pellet": ["Pellet"], "Soluble": ["Soluble"]}

    file_name = []

    scan_type = []
    scan_number = []
    retention_time = []
    cast_spectra = []

    mz_value = []

    for raw_name in files:
        raw = RawFile(raw_name)
        print(raw_name)
        for i in tqdm(range(1, raw.number_of_scans), desc="Processing scans", ncols=100):
            raw_scan = Scan.from_file(raw._raw_file_access, scan_number=i)
            file_name.append(raw_name)

            if str(helper_regex(raw_scan.scan_type)) == 'ms':
                scan_type.append('MS1')
                scan_number.append(raw_scan.scan_statistics.scan_number)
                retention_time.append(raw.get_retention_time_from_scan_number(raw_scan.scan_statistics.scan_number))
                mz_value.append('')

                data_intensities = [0]*13690
                scan_masses = raw_scan.preferred_masses
                scan_intensities = raw_scan.preferred_intensities

                for j in range(0,len(scan_masses)):
                    index = int(round(scan_masses[j], 2)*10)
                    if index > 6000 and index < 19360:
                        data_intensities[index-6000] = scan_intensities[j] + data_intensities[index-6000]

                cast_spectra.append(data_intensities)


            if str(helper_regex(raw_scan.scan_type)) == 'ms2':
                scan_type.append('MS2')
                scan_number.append(raw_scan.scan_statistics.scan_number)
                retention_time.append(raw.get_retention_time_from_scan_number(raw_scan.scan_statistics.scan_number))
                mz_value.append(float(re.findall(r'[\d]*[.][\d]+', raw_scan.scan_type)[1]))

                data_intensities = [0]*1600
                scan_masses = raw_scan.preferred_masses
                scan_intensities = raw_scan.preferred_intensities

                for j in range(0,len(scan_masses)):
                    index = round(scan_masses[j])
                    if index > 400 and index < 2000:
                        data_intensities[index-400] = scan_intensities[j] + data_intensities[index-400]
                data_intensities = np.array(data_intensities)
                max_value = np.max(data_intensities)
                data_intensities_norm = data_intensities / max_value
                data_intensities_norm = data_intensities_norm.astype(np.float16)
                data_intensities_norm.tolist()
                cast_spectra.append(data_intensities_norm)

    scan_dict = {'sample_name': file_name, 'scan': scan_number,'scan_type': scan_type, 'retntion time': retention_time, 'm/z': mz_value, 'cast spectra': cast_spectra}

    with open(cast_path, "wb") as f:
        pickle.dump(scan_dict, f)

    return()

def ID_import(tdportal, databank, cast_path):
  def str_to_int(st):
      internal = []
      digits = re.findall(r'\d+', st)
      for i in range(0, len(digits)):
          internal.append(int(digits[i]))
      return(internal)

  scan_number = [0]*len(tdportal['File Name'])
  td_samples = []

  for i in range(0, len(tdportal['File Name'])):
      scan_number[i] = str_to_int(str(tdportal['Fragment Scans'][i]))
      if tdportal['File Name'][i] not in td_samples:
        td_samples.append(tdportal['File Name'][i])

  my_dic_scan = {key: [] for key in td_samples}
  my_dic_index = {key: [] for key in td_samples}

  for i in range(0, len(tdportal['File Name'])):
      my_dic_scan[tdportal['File Name'][i]].append(scan_number[i])
      my_dic_index[tdportal['File Name'][i]].append([i]*len(scan_number[i]))

  for i in range(0, len(td_samples)):
      nested_list = my_dic_scan[td_samples[i]]
      flat_list = []
      for item in nested_list:
          if isinstance(item, list):
              flat_list.extend(item)
          else:
              flat_list.append(item)
      my_dic_scan[td_samples[i]] = [elem for sublist in flat_list for elem in (sublist if isinstance(sublist, list) else [sublist])]


  for i in range(0, len(td_samples)):
      nested_list = my_dic_index[td_samples[i]]
      flat_list = []
      for item in nested_list:
          if isinstance(item, list):
              flat_list.extend(item)
          else:
              flat_list.append(item)
      my_dic_index[td_samples[i]] = [elem for sublist in flat_list for elem in (sublist if isinstance(sublist, list) else [sublist])]

  sequence = []
  MASS = []
  Uniprot_ID = []
  Accession = []

  for i in tqdm(range(0, len(databank['scan'])), desc="Processing scans", ncols=100):
      if databank['scan'][i] in my_dic_scan[databank['sample_name'][i]]:
          tt = my_dic_index[databank['sample_name'][i]][my_dic_scan[databank['sample_name'][i]].index(databank['scan'][i])]
          sequence.append(tdportal['Sequence'][tt])
          MASS.append(tdportal['Average Mass'][tt])
          Uniprot_ID.append(tdportal['Uniprot Id'][tt])
          Accession.append(tdportal['Accession'][tt])
      else:
          sequence.append('None')
          MASS.append('None')
          Uniprot_ID.append('None')
          Accession.append('None')


  databank['sequence'] = sequence
  databank['MASS'] = MASS
  databank['Uniprot ID'] = Uniprot_ID
  databank['Accession'] = Accession

  databank = pd.DataFrame(databank)

  databank.to_hdf(cast_path, key="databank", mode="w")

  return()


#Build Databank

In [ ]:
wholeCasting("D:/joker/joker1162025/Samples/",'D:/joker/databank')

with open("D:/joker/databank", "rb") as f:
    databank = pickle.load(f)
tdportal = pd.read_csv('D:/joker/hit_report.csv')
cast_path = 'D:/joker/databank_updated'

ID_import(tdportal, databank, cast_path)

#Align retention times

In [ ]:

df = pd.read_hdf("D:/joker/databank_updated", key="databank")
sample_list = df['sample_name'].unique()

def engine(cast1, cast2):
    X = [cast1, cast2]
    return(round(cosine_similarity(X)[1][0], 2))

ret_1 = []
ret_2 = []

df = df[df['scan_type'] == 'MS2'].reset_index(drop=True)
df1 = df[df['sample_name'] == sample_list[0]].reset_index(drop=True)
df2 = df[df['sample_name'] == sample_list[2]].reset_index(drop=True)

for i in range(len(df1)):
    print( i, 'from', len(df1))
    for j in range(len(df2)):
        if df1['m/z'][i] - 0.1 < df2['m/z'][j] < df1['m/z'][i] + 0.1:
            a = engine(df1['cast spectra'][i], df2['cast spectra'][j])
            if a > 0.99:
                ret_1.append(df1['retntion time'][i])
                ret_2.append(df2['retntion time'][j])

# Zip the two lists together
zipped = list(zip(ret_1, ret_2))

# Sort by the first element (ret_1)
zipped_sorted = sorted(zipped, key=lambda x: x[0])

# Unzip back into separate lists
ret_1_sorted, ret_2_sorted = zip(*zipped_sorted)

# Optional: convert back to lists
ret_1_sorted = list(ret_1_sorted)
ret_2_sorted = list(ret_2_sorted)

# Convert lists to pandas Series
ret_1_series = pd.Series(ret_1_sorted)
ret_2_series = pd.Series(ret_2_sorted)

# Apply rolling average with window of 20
ret_1_smooth = ret_1_series.rolling(window=10, center=True).mean()
ret_2_smooth = ret_2_series.rolling(window=10, center=True).mean()

plt.figure(figsize=(8, 6))
plt.plot(ret_1_smooth, ret_2_smooth, '.', alpha=0.7, color='mediumseagreen')
plt.xlabel(sample_list[0])
plt.ylabel(sample_list[2])
#plt.title('Smoothed Retention Time Comparison')
plt.grid(True)
#plt.tight_layout()
plt.show()

# Make sure ret_1_smooth and ret_2_smooth are numpy arrays and drop NaNs
valid = (~pd.isna(ret_1_smooth)) & (~pd.isna(ret_2_smooth))
x_vals = np.array(ret_1_smooth[valid])
y_vals = np.array(ret_2_smooth[valid])

# Create interpolation function (linear, but you can choose 'cubic' etc.)
interpolator = interp1d(x_vals, y_vals, kind='linear', fill_value="extrapolate")

def get_ret2_from_ret1(x):
    return interpolator(x)

x_query = 100  # or a list/array of x values
y_result = get_ret2_from_ret1(x_query)
print(f"Interpolated value at x={x_query}: y={y_result}")


#Find unique features

In [ ]:
df = df[df['scan_type']=='MS2']

mzvalues = df['m/z']
retvalues = df['retntion time']
idvalues = df['Uniprot ID']
massvalues = df['MASS']

# Zip the two lists together
zipped = list(zip(retvalues, mzvalues, idvalues, massvalues))

# Sort by the first element (ret_1)
zipped_sorted = sorted(zipped, key=lambda x: x[0])

# Unzip back into separate lists
ret_sorted, mz_sorted, id_sorted, mass_sorted = zip(*zipped_sorted)

# Optional: convert back to lists
ret_sorted = list(ret_sorted)
mz_sorted = list(mz_sorted)
id_sorted = list(id_sorted)
mass_sorted = list(mass_sorted)

tolerance = 1
window = 10
feature_mz = []
feature_ret = []
feature_id = []
feature_mass = []

for i in range(0, len(ret_sorted)):
    print(i, 'from', len(ret_sorted))
    if 600 < mz_sorted[i] < 1396 and ret_sorted[i] > 12:
        indexes = [index for index, value in enumerate(feature_mz) if abs(value - mz_sorted[i]) <= tolerance]
        counter = 0
        for j in indexes:
            if abs(feature_ret[j] - ret_sorted[i]) <= window:
                counter = counter + 1
        if counter == 0:
            print('unique')
            feature_mz.append(mz_sorted[i])
            feature_ret.append(ret_sorted[i])
            feature_id.append(id_sorted[i])
            feature_mass.append(mass_sorted[i])

#Quantification

In [ ]:
samples = sample_list
df = pd.read_hdf("D:/joker/databank_updated", key="databank")

def quantification(df, sample_number):
    df_all = df.copy()
    samples = df_all['sample_name'].unique()
    df = df_all[df_all['scan_type'] == 'MS1'].reset_index(drop=True)
    df_sample = df[df['sample_name'] == samples[sample_number]].reset_index(drop=True)
    results = []

    for i in range(0, len(feature_mz)):
        print( i, 'from', len(feature_mz))
        a = []
        for j in range(0, len(df_sample)):
            if feature_ret[i] - 4 < df_sample['retntion time'][j] < feature_ret[i] + 4:
                a.append(df_sample['cast spectra'][j][int(round(feature_mz[i], 2)*10) - 6000])
        results.append(np.mean(a))
    return(results)

#CSV output

In [ ]:
df_all = df.copy()

df1 = df_all[df_all['scan_type'] == 'MS1'].reset_index(drop=True)
samples = df1['sample_name'].unique()

results = []
for i in range(0, len(samples)):
    results.append(quantification(df, i))

output = pd.DataFrame(results)

merged = [f"{a}:{b}:{c}" for a, b, c in zip(feature_id, feature_mass, feature_mz)]

output.columns= merged

output.insert(0, 'sample_name', samples)

output.to_csv('D:/joker/result.csv')

#Large scale visualization

In [ ]:

for i in range(0, 6):
    for j in range(0, 6):
        name = str(i)+str(j)+'.jpeg'

        results_1 = quantification(df, i)
        results_2 = quantification(df, j)

        # Create the scatter plot
        plt.scatter(results_1, results_2, label='Data points')

        # Perform linear regression
        slope, intercept, r_value, p_value, std_err = linregress(results_1, results_2)

        # Create the linear regression line
        regression_line = np.array(results_1) * slope + intercept

        # Plot the regression line
        plt.plot(results_1, regression_line, color='red', label=f'Linear fit: $R^2$ = {r_value**2:.2f}')

        # Add labels and a title
        plt.xlabel(samples[i])  # Label for the x-axis
        plt.ylabel(samples[j])  # Label for the y-axis
        #plt.title('Quantification with Linear Regression')  # Title of the plot

        # Show the legend
        plt.legend()
        plt.savefig(name)
        # Show the plot
        plt.show()